### This is just quick to see if I can get a naiive bayes classifier up and running for datasets

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns


#from sklearn.preprocessing import Imputer

### The next few lines are just for construction of the original df. I wrote the df into a .csv file so you can start things off a couple of cells down

In [2]:
os.chdir(r'C:\Users\timdaugird\Google Drive\legant_lab\_projects\200217_adipogen_zstacks\cell_profiler_output\dcts_output\segm_img_tests\in_out_csvs')

#os.listdir()

'''list of csv files to be read in'''
csv_lst = ['instad438-xy07.csv',
 'in_stad436-xy01.csv',
 'in_stad440-xy10.csv',
 'in_stad441-xy01.csv',
 'in_stad444-xy02.csv',
 'in_stad445-xy04.csv',
 'in_stad445-xy06.csv',
 'in_stad445-xy09.csv',
 'in_stad446-xy04.csv',
 'in_stad446-xy07.csv',
 'in_stad446-xy09.csv',
 'in_stad446-xy10.csv',
 'in_stad447-xy03.csv',
 'in_stad447-xy07.csv',
 'in_stad447-xy09.csv',
 'outstad438-xy07.csv',
           'out_stad436-xy01.csv',
 'out_stad440-xy10.csv',
 'out_stad441-xy01.csv',
 'out_stad444-xy02.csv',
 'out_stad445-xy04.csv',
 'out_stad445-xy06.csv',
 'out_stad445-xy09.csv',
 'out_stad446-xy04.csv',
 'out_stad446-xy07.csv',
 'out_stad446-xy09.csv',
 'out_stad446-xy10.csv',
 'out_stad447-xy03.csv',
 'out_stad447-xy07.csv',
 'out_stad447-xy09.csv']

'''read in all of the .csv files and concatenate into one big df'''

df_lst = []

for i in csv_lst:
    df_lst.append(pd.read_csv(i))
df_whole = pd.concat(df_lst)

In [5]:
'''Just to make sure that I didnt miss some small object in the manual curation I will turn everything less than 10000 to the out category '''
df_in = df_whole.loc[df_whole.in_object == 1]
df_out_ = df_whole.loc[df_whole.in_object == 0]

'''pull out smalle values in the df_in df'''
df_in_change = df_in.loc[df_in.AreaShape_Area < 20000] #these are the wrong attributed entries in df_in
df_in = df_in.loc[df_in.AreaShape_Area >= 20000] #these are the right entries for df_in

df_in_change.in_object = np.zeros(len(df_in_change)) #switch in_object for all of the wrong entries
df_out = pd.concat([df_out_, df_in_change]) #move the wrong entries from df_in to df_out

df_whole = pd.concat([df_in, df_out]) #make one big dataframe again

In [6]:



'''double check that the above rearrangement worked as planned'''
df_ = df_whole.loc[df_whole.AreaShape_Area < 20000]
np.sum(df_.in_object)

0.0

In [2]:
os.chdir(r'C:\Users\timdaugird\Google Drive\legant_lab\_projects\200217_adipogen_zstacks\cell_profiler_output\dcts_output')

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'classifier.ipynb',
 'data_for_nb_training.csv',
 'df1.csv',
 'df2.csv',
 'img_ls.csv',
 'MyExpt_Experiment.csv',
 'MyExpt_Image.csv',
 'MyExpt_nuclei.csv',
 'segm_exp_analysis_nb_parsed.ipynb',
 'segm_img_tests',
 'segm_parse_testing.ipynb']

In [4]:
df_whole = pd.read_csv('data_for_nb_training.csv')

In [5]:
list(df_whole)

['Unnamed: 0',
 'ImageNumber',
 'ObjectNumber',
 'AreaShape_Area',
 'AreaShape_Center',
 'AreaShape_Center.1',
 'AreaShape_Center.2',
 'AreaShape_Compactness',
 'AreaShape_Eccentricity',
 'AreaShape_EulerNumber',
 'AreaShape_Extent',
 'AreaShape_FormFactor',
 'AreaShape_MajorAxisLength',
 'AreaShape_MaxFeretDiameter',
 'AreaShape_MaximumRadius',
 'AreaShape_MeanRadius',
 'AreaShape_MedianRadius',
 'AreaShape_MinFeretDiameter',
 'AreaShape_MinorAxisLength',
 'AreaShape_Orientation',
 'AreaShape_Perimeter',
 'AreaShape_Solidity',
 'Intensity_IntegratedIntensityEdge_dapi',
 'Intensity_IntegratedIntensity_dapi',
 'Intensity_LowerQuartileIntensity_dapi',
 'Intensity_MADIntensity_dapi',
 'Intensity_MassDisplacement_dapi',
 'Intensity_MaxIntensityEdge_dapi',
 'Intensity_MaxIntensity_dapi',
 'Intensity_MeanIntensityEdge_dapi',
 'Intensity_MeanIntensity_dapi',
 'Intensity_MedianIntensity_dapi',
 'Intensity_MinIntensityEdge_dapi',
 'Intensity_MinIntensity_dapi',
 'Intensity_StdIntensityEdge_dapi

In [6]:
len(df_whole_)

376

In [7]:
len(df_whole)

376

### this is almost entirely taken from https://heartbeat.fritz.ai/naive-bayes-classifier-in-python-using-scikit-learn-13c4deb83bcf

### Start off with some preprocessing steps

In [13]:
'''split the data into train and target sets. In this caes x will be the train data and y will be the target parameter'''
df = df_whole.copy() # make a copy of original df. I am going to alter this a little bit in order to pull out train and test cats

target_cat_name = 'in_object'

'''remove unwanted categories'''
unwanted_cat = ['Unnamed: 0']#, 'ImageNumber', 'ObjectNumber']
df.drop(unwanted_cat, axis = 1, inplace = True)

'''define train and target data'''
x = df.drop('in_object', axis = 1)
y = df[target_cat_name]

In [14]:
df.head()

,ImageNumber,ObjectNumber,AreaShape_Area,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_Center_Z,AreaShape_Compactness,AreaShape_Eccentricity,AreaShape_EulerNumber,AreaShape_Extent,...,Location_MaxIntensity_Y_c2,Location_MaxIntensity_Y_c3,Location_MaxIntensity_Y_cebp,Location_MaxIntensity_Y_dapi,Location_MaxIntensity_Z_c2,Location_MaxIntensity_Z_c3,Location_MaxIntensity_Z_cebp,Location_MaxIntensity_Z_dapi,Number_Object_Number,in_object
0,1,1,8331,282.0,0.0,1.0,2.352487,0.968781,1.0,0.658577,...,3.0,21.0,5.0,4.0,0.0,0.0,0.0,0.0,1,0.0
1,1,2,49529,2043.0,89.0,1.0,1.809739,0.919641,1.0,0.578779,...,45.0,52.0,186.0,376.0,0.0,0.0,0.0,0.0,2,0.0
2,1,3,54217,1186.0,186.0,1.0,1.018289,0.560011,1.0,0.755912,...,226.0,291.0,213.0,226.0,0.0,0.0,0.0,0.0,3,1.0
3,1,4,46013,455.0,474.0,1.0,1.013055,0.511901,1.0,0.776630,...,364.0,456.0,426.0,513.0,0.0,0.0,0.0,0.0,4,1.0
4,1,5,38938,1336.0,473.0,1.0,1.025519,0.598589,1.0,0.749168,...,489.0,437.0,471.0,531.0,0.0,0.0,0.0,0.0,5,1.0


In [15]:
'''check for missing values'''
df.isnull().sum()

ImageNumber                     0
ObjectNumber                    0
AreaShape_Area                  0
AreaShape_Center_X              0
AreaShape_Center_Y              0
                               ..
Location_MaxIntensity_Z_c3      0
Location_MaxIntensity_Z_cebp    0
Location_MaxIntensity_Z_dapi    0
Number_Object_Number            0
in_object                       0
Length: 110, dtype: int64

In [18]:
'''If I had run into some null values then I could use the following'''
#from sklearn.preprocessing import Imputer
#imputer = Imputer(missing_values = "NaN", strategy = "mean", axis = 0)
#imputer = imputer.fit(x[:,:])
#x[:, 1:3] = imputer.transform(x[:, 1:3])

'If I had run into some null values then I could use the following'

In [12]:
'''check uniform data types'''
df.dtypes

Unnamed: 0                                  int64
AreaShape_Area                              int64
AreaShape_Center_X                        float64
AreaShape_Center_Y                        float64
AreaShape_Center_Z                        float64
AreaShape_Compactness                     float64
AreaShape_Eccentricity                    float64
AreaShape_EulerNumber                     float64
AreaShape_Extent                          float64
AreaShape_FormFactor                      float64
AreaShape_MajorAxisLength                 float64
AreaShape_MaxFeretDiameter                float64
AreaShape_MaximumRadius                   float64
AreaShape_MeanRadius                      float64
AreaShape_MedianRadius                    float64
AreaShape_MinFeretDiameter                float64
AreaShape_MinorAxisLength                 float64
AreaShape_MinorAxisLength.1               float64
AreaShape_MinorAxisLength.2               float64
AreaShape_MinorAxisLength.3               float64


In [16]:
type(x)

pandas.core.frame.DataFrame

In [17]:
type(x_train)

numpy.ndarray

In [6]:
'''split data into train and test sets'''
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .25, random_state = 12)

In [7]:
'''this is a feature scaling step. This will standardize all of the data in order to pull everything into the same range'''

sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test)

In [8]:
'''set up gaussian naive bayes model and fit it to the data'''
model = GaussianNB()
model.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [9]:
'''make predictions based on the trained data'''
y_pred = model.predict(x_test)


In [10]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)


In [11]:
accuracy

0.9042553191489362

In [12]:
precision

0.9607843137254902

In [23]:
confusion_matrix(y_test, y_pred)

array([[32,  5],
       [ 5, 52]], dtype=int64)

### A little for shits and giggles I would as well to see if I can get better fit through using more categories


### this did end up working a bit better so I think that I will use that classifier from here on. I went ahead wrote the df with the additional categories into a .csv file for future use

In [7]:
os.chdir(r'C:\Users\timdaugird\Google Drive\legant_lab\_projects\200217_adipogen_zstacks\cell_profiler_output\dcts_output')
df_orig = pd.read_csv('MyExpt_nuclei.csv')

In [9]:
df_final = df_orig.loc[(df_orig.ImageNumber.isin(df_whole.ImageNumber)) & df_orig.ObjectNumber.isin(df_whole.ObjectNumber)]

In [19]:
list(df_final)

['ImageNumber',
 'ObjectNumber',
 'AreaShape_Area',
 'AreaShape_Center_X',
 'AreaShape_Center_Y',
 'AreaShape_Center_Z',
 'AreaShape_Compactness',
 'AreaShape_Eccentricity',
 'AreaShape_EulerNumber',
 'AreaShape_Extent',
 'AreaShape_FormFactor',
 'AreaShape_MajorAxisLength',
 'AreaShape_MaxFeretDiameter',
 'AreaShape_MaximumRadius',
 'AreaShape_MeanRadius',
 'AreaShape_MedianRadius',
 'AreaShape_MinFeretDiameter',
 'AreaShape_MinorAxisLength',
 'AreaShape_Orientation',
 'AreaShape_Perimeter',
 'AreaShape_Solidity',
 'Intensity_IntegratedIntensityEdge_c2',
 'Intensity_IntegratedIntensityEdge_c3',
 'Intensity_IntegratedIntensityEdge_cebp',
 'Intensity_IntegratedIntensityEdge_dapi',
 'Intensity_IntegratedIntensity_c2',
 'Intensity_IntegratedIntensity_c3',
 'Intensity_IntegratedIntensity_cebp',
 'Intensity_IntegratedIntensity_dapi',
 'Intensity_LowerQuartileIntensity_c2',
 'Intensity_LowerQuartileIntensity_c3',
 'Intensity_LowerQuartileIntensity_cebp',
 'Intensity_LowerQuartileIntensity_da

In [21]:
in_object_array

0    0.0
1    0.0
0    1.0
1    1.0
2    1.0
    ... 
4    1.0
0    0.0
5    1.0
1    0.0
6    1.0
Name: in_object, Length: 376, dtype: float64

In [22]:
df_final.sort_values(['ImageNumber', 'ObjectNumber'], inplace = True)
df_whole.sort_values(['ImageNumber', 'ObjectNumber'], inplace = True)

in_object_array = df_whole.in_object

df_final['in_object'] = np.array(in_object_array)

C:\Users\timdaugird\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\timdaugird\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
len(df_whole)

376

In [6]:
df_orig.head()

,ImageNumber,ObjectNumber,AreaShape_Area,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_Center_Z,AreaShape_Compactness,AreaShape_Eccentricity,AreaShape_EulerNumber,AreaShape_Extent,...,Location_MaxIntensity_X_dapi,Location_MaxIntensity_Y_c2,Location_MaxIntensity_Y_c3,Location_MaxIntensity_Y_cebp,Location_MaxIntensity_Y_dapi,Location_MaxIntensity_Z_c2,Location_MaxIntensity_Z_c3,Location_MaxIntensity_Z_cebp,Location_MaxIntensity_Z_dapi,Number_Object_Number
0,1,1,8331,282.0,0.0,1.0,2.352487,0.968781,1.0,0.658577,...,351.0,3.0,21.0,5.0,4.0,0.0,0.0,0.0,0.0,1
1,1,2,49529,2043.0,89.0,1.0,1.809739,0.919641,1.0,0.578779,...,2043.0,45.0,52.0,186.0,376.0,0.0,0.0,0.0,0.0,2
2,1,3,54217,1186.0,186.0,1.0,1.018289,0.560011,1.0,0.755912,...,1121.0,226.0,291.0,213.0,226.0,0.0,0.0,0.0,0.0,3
3,1,4,46013,455.0,474.0,1.0,1.013055,0.511901,1.0,0.776630,...,511.0,364.0,456.0,426.0,513.0,0.0,0.0,0.0,0.0,4
4,1,5,38938,1336.0,473.0,1.0,1.025519,0.598589,1.0,0.749168,...,1286.0,489.0,437.0,471.0,531.0,0.0,0.0,0.0,0.0,5


In [7]:
list(df_whole)

['Unnamed: 0',
 'ImageNumber',
 'ObjectNumber',
 'AreaShape_Area',
 'AreaShape_Compactness',
 'AreaShape_Eccentricity',
 'AreaShape_EulerNumber',
 'AreaShape_Extent',
 'AreaShape_FormFactor',
 'AreaShape_MajorAxisLength',
 'AreaShape_MaxFeretDiameter',
 'AreaShape_MaximumRadius',
 'AreaShape_MeanRadius',
 'AreaShape_MedianRadius',
 'AreaShape_MinFeretDiameter',
 'AreaShape_MinorAxisLength',
 'AreaShape_Orientation',
 'AreaShape_Perimeter',
 'AreaShape_Solidity',
 'in_object']

In [8]:
'''merge the data frames'''

#df_merged = pd.merge(df_orig, df_whole, on = [ 'ImageNumber',
# 'ObjectNumber',
# 'AreaShape_Area',
# 'AreaShape_Compactness',
# 'AreaShape_Eccentricity',
# 'AreaShape_EulerNumber',
# 'AreaShape_Extent',
# 'AreaShape_FormFactor',
# 'AreaShape_MajorAxisLength',
# 'AreaShape_MaxFeretDiameter',
# 'AreaShape_MaximumRadius',
# 'AreaShape_MeanRadius',
# 'AreaShape_MedianRadius',
# 'AreaShape_MinFeretDiameter',
# 'AreaShape_MinorAxisLength',
# 'AreaShape_Orientation',
# 'AreaShape_Perimeter',
# 'AreaShape_Solidity',], how = 'inner')

In [7]:
df_merged = pd.merge(df_orig, df_whole, on = [ 'ImageNumber', 'ObjectNumber'], how = 'inner')

In [8]:
list(df_merged)

['ImageNumber',
 'ObjectNumber',
 'AreaShape_Area_x',
 'AreaShape_Center_X',
 'AreaShape_Center_Y',
 'AreaShape_Center_Z',
 'AreaShape_Compactness_x',
 'AreaShape_Eccentricity_x',
 'AreaShape_EulerNumber_x',
 'AreaShape_Extent_x',
 'AreaShape_FormFactor_x',
 'AreaShape_MajorAxisLength_x',
 'AreaShape_MaxFeretDiameter_x',
 'AreaShape_MaximumRadius_x',
 'AreaShape_MeanRadius_x',
 'AreaShape_MedianRadius_x',
 'AreaShape_MinFeretDiameter_x',
 'AreaShape_MinorAxisLength_x',
 'AreaShape_Orientation_x',
 'AreaShape_Perimeter_x',
 'AreaShape_Solidity_x',
 'Intensity_IntegratedIntensityEdge_c2',
 'Intensity_IntegratedIntensityEdge_c3',
 'Intensity_IntegratedIntensityEdge_cebp',
 'Intensity_IntegratedIntensityEdge_dapi',
 'Intensity_IntegratedIntensity_c2',
 'Intensity_IntegratedIntensity_c3',
 'Intensity_IntegratedIntensity_cebp',
 'Intensity_IntegratedIntensity_dapi',
 'Intensity_LowerQuartileIntensity_c2',
 'Intensity_LowerQuartileIntensity_c3',
 'Intensity_LowerQuartileIntensity_cebp',
 'Int

In [9]:
df_merged['AreaShape_Area_x'] == df_merged['AreaShape_Area_y']

0      True
1      True
2      True
3      True
4      True
       ... 
371    True
372    True
373    True
374    True
375    True
Length: 376, dtype: bool

In [10]:
len(df_whole)

376

### right now a couple of the categories are duplicated so I will need to take care of that

In [31]:
list(df_merged)

['ImageNumber',
 'ObjectNumber',
 'AreaShape_Area_x',
 'AreaShape_Center_X',
 'AreaShape_Center_Y',
 'AreaShape_Center_Z',
 'AreaShape_Compactness_x',
 'AreaShape_Eccentricity_x',
 'AreaShape_EulerNumber_x',
 'AreaShape_Extent_x',
 'AreaShape_FormFactor_x',
 'AreaShape_MajorAxisLength_x',
 'AreaShape_MaxFeretDiameter_x',
 'AreaShape_MaximumRadius_x',
 'AreaShape_MeanRadius_x',
 'AreaShape_MedianRadius_x',
 'AreaShape_MinFeretDiameter_x',
 'AreaShape_MinorAxisLength_x',
 'AreaShape_Orientation_x',
 'AreaShape_Perimeter_x',
 'AreaShape_Solidity_x',
 'Intensity_IntegratedIntensityEdge_c2',
 'Intensity_IntegratedIntensityEdge_c3',
 'Intensity_IntegratedIntensityEdge_cebp',
 'Intensity_IntegratedIntensityEdge_dapi',
 'Intensity_IntegratedIntensity_c2',
 'Intensity_IntegratedIntensity_c3',
 'Intensity_IntegratedIntensity_cebp',
 'Intensity_IntegratedIntensity_dapi',
 'Intensity_LowerQuartileIntensity_c2',
 'Intensity_LowerQuartileIntensity_c3',
 'Intensity_LowerQuartileIntensity_cebp',
 'Int

In [10]:
cats_to_keep = ['ImageNumber',
 'ObjectNumber',
 'AreaShape_Area_x',
 'AreaShape_Center_X',
 'AreaShape_Center_Y',
 'AreaShape_Center_Z',
 'AreaShape_Compactness_x',
 'AreaShape_Eccentricity_x',
 'AreaShape_EulerNumber_x',
 'AreaShape_Extent_x',
 'AreaShape_FormFactor_x',
 'AreaShape_MajorAxisLength_x',
 'AreaShape_MaxFeretDiameter_x',
 'AreaShape_MaximumRadius_x',
 'AreaShape_MeanRadius_x',
 'AreaShape_MedianRadius_x',
 'AreaShape_MinFeretDiameter_x',
 'AreaShape_MinorAxisLength_x',
 'AreaShape_Orientation_x',
 'AreaShape_Perimeter_x',
 'AreaShape_Solidity_x',
 'Intensity_IntegratedIntensityEdge_dapi',
 'Intensity_IntegratedIntensity_dapi',
 'Intensity_LowerQuartileIntensity_dapi',
 'Intensity_MADIntensity_dapi',
 'Intensity_MassDisplacement_dapi',
 'Intensity_MaxIntensityEdge_dapi',
 'Intensity_MaxIntensity_dapi',
 'Intensity_MeanIntensityEdge_dapi',
 'Intensity_MeanIntensity_dapi',
 'Intensity_MedianIntensity_dapi',
 'Intensity_MinIntensityEdge_dapi',
 'Intensity_MinIntensity_dapi',
 'Intensity_StdIntensityEdge_dapi',
 'Intensity_StdIntensity_dapi',
 'Intensity_UpperQuartileIntensity_dapi',
 'Location_CenterMassIntensity_X_dapi',
 'Location_CenterMassIntensity_Y_dapi',
 'Location_CenterMassIntensity_Z_dapi',
 'Location_Center_X',
 'Location_Center_Y',
 'Location_Center_Z',
 'Location_MaxIntensity_X_dapi',
 'Location_MaxIntensity_Y_dapi',
 'Location_MaxIntensity_Z_dapi',
               'in_object']

In [23]:
cats_to_keep = ['ImageNumber',
 'ObjectNumber',
 'AreaShape_Area',
 'AreaShape_Center_X',
 'AreaShape_Center_Y',
 'AreaShape_Center_Z',
 'AreaShape_Compactness',
 'AreaShape_Eccentricity',
 'AreaShape_EulerNumber',
 'AreaShape_Extent',
 'AreaShape_FormFactor',
 'AreaShape_MajorAxisLength',
 'AreaShape_MaxFeretDiameter',
 'AreaShape_MaximumRadius',
 'AreaShape_MeanRadius',
 'AreaShape_MedianRadius',
 'AreaShape_MinFeretDiameter',
 'AreaShape_MinorAxisLength',
 'AreaShape_Orientation',
 'AreaShape_Perimeter',
 'AreaShape_Solidity',
 'Intensity_IntegratedIntensityEdge_dapi',
 'Intensity_IntegratedIntensity_dapi',
 'Intensity_LowerQuartileIntensity_dapi',
 'Intensity_MADIntensity_dapi',
 'Intensity_MassDisplacement_dapi',
 'Intensity_MaxIntensityEdge_dapi',
 'Intensity_MaxIntensity_dapi',
 'Intensity_MeanIntensityEdge_dapi',
 'Intensity_MeanIntensity_dapi',
 'Intensity_MedianIntensity_dapi',
 'Intensity_MinIntensityEdge_dapi',
 'Intensity_MinIntensity_dapi',
 'Intensity_StdIntensityEdge_dapi',
 'Intensity_StdIntensity_dapi',
 'Intensity_UpperQuartileIntensity_dapi',
 'Location_CenterMassIntensity_X_dapi',
 'Location_CenterMassIntensity_Y_dapi',
 'Location_CenterMassIntensity_Z_dapi',
 'Location_Center_X',
 'Location_Center_Y',
 'Location_Center_Z',
 'Location_MaxIntensity_X_dapi',
 'Location_MaxIntensity_Y_dapi',
 'Location_MaxIntensity_Z_dapi',
               'in_object']

In [24]:
df_jnk = df_final[cats_to_keep]

### I would like to clean up the feature names in the .csv file so that it makes future stuff a bit easier

In [17]:
list(df_jnk)

['ImageNumber',
 'ObjectNumber',
 'AreaShape_Area_x',
 'AreaShape_Center_X',
 'AreaShape_Center_Y',
 'AreaShape_Center_Z',
 'AreaShape_Compactness_x',
 'AreaShape_Eccentricity_x',
 'AreaShape_EulerNumber_x',
 'AreaShape_Extent_x',
 'AreaShape_FormFactor_x',
 'AreaShape_MajorAxisLength_x',
 'AreaShape_MaxFeretDiameter_x',
 'AreaShape_MaximumRadius_x',
 'AreaShape_MeanRadius_x',
 'AreaShape_MedianRadius_x',
 'AreaShape_MinFeretDiameter_x',
 'AreaShape_MinorAxisLength_x',
 'AreaShape_Orientation_x',
 'AreaShape_Perimeter_x',
 'AreaShape_Solidity_x',
 'Intensity_IntegratedIntensityEdge_dapi',
 'Intensity_IntegratedIntensity_dapi',
 'Intensity_LowerQuartileIntensity_dapi',
 'Intensity_MADIntensity_dapi',
 'Intensity_MassDisplacement_dapi',
 'Intensity_MaxIntensityEdge_dapi',
 'Intensity_MaxIntensity_dapi',
 'Intensity_MeanIntensityEdge_dapi',
 'Intensity_MeanIntensity_dapi',
 'Intensity_MedianIntensity_dapi',
 'Intensity_MinIntensityEdge_dapi',
 'Intensity_MinIntensity_dapi',
 'Intensity_St

In [34]:
cat_renames = ['Unnamed: 0',
 'AreaShape_Area',
 'AreaShape_Center_X',
 'AreaShape_Center_Y',
 'AreaShape_Center_Z',
 'AreaShape_Compactness',
 'AreaShape_Eccentricity',
 'AreaShape_EulerNumber',
 'AreaShape_Extent',
 'AreaShape_FormFactor',
 'AreaShape_MajorAxisLength',
 'AreaShape_MaxFeretDiameter',
 'AreaShape_MaximumRadius',
 'AreaShape_MeanRadius',
 'AreaShape_MedianRadius',
 'AreaShape_MinFeretDiameter',
 'AreaShape_MinorAxisLength',
 'AreaShape_Orientation',
 'AreaShape_Perimeter',
 'AreaShape_Solidity',
 'Intensity_IntegratedIntensityEdge_dapi',
 'Intensity_IntegratedIntensity_dapi',
 'Intensity_LowerQuartileIntensity_dapi',
 'Intensity_MADIntensity_dapi',
 'Intensity_MassDisplacement_dapi',
 'Intensity_MaxIntensityEdge_dapi',
 'Intensity_MaxIntensity_dapi',
 'Intensity_MeanIntensityEdge_dapi',
 'Intensity_MeanIntensity_dapi',
 'Intensity_MedianIntensity_dapi',
 'Intensity_MinIntensityEdge_dapi',
 'Intensity_MinIntensity_dapi',
 'Intensity_StdIntensityEdge_dapi',
 'Intensity_StdIntensity_dapi',
 'Intensity_UpperQuartileIntensity_dapi',
 'Location_CenterMassIntensity_X_dapi',
 'Location_CenterMassIntensity_Y_dapi',
 'Location_CenterMassIntensity_Z_dapi',
 'Location_Center_X',
 'Location_Center_Y',
 'Location_Center_Z',
 'Location_MaxIntensity_X_dapi',
 'Location_MaxIntensity_Y_dapi',
 'Location_MaxIntensity_Z_dapi',
 'in_object']

In [18]:
'''I need to create a dictionary for renaming things'''
rename_dict = {}

orig_names_lst = [ 'AreaShape_Area_x',
 'AreaShape_Center_X',
 'AreaShape_Center_Y',
 'AreaShape_Center_Z',
 'AreaShape_Compactness_x',
 'AreaShape_Eccentricity_x',
 'AreaShape_EulerNumber_x',
 'AreaShape_Extent_x',
 'AreaShape_FormFactor_x',
 'AreaShape_MajorAxisLength_x',
 'AreaShape_MaxFeretDiameter_x',
 'AreaShape_MaximumRadius_x',
 'AreaShape_MeanRadius_x',
 'AreaShape_MedianRadius_x',
 'AreaShape_MinFeretDiameter_x',
 'AreaShape_MinorAxisLength_x',
 'AreaShape_Orientation_x',
 'AreaShape_Perimeter_x',
 'AreaShape_Solidity_x']

for i in orig_names_lst:
    rename_dict.update({i : i[:-2]})

In [35]:
rename_dict

{'AreaShape_Area_x': 'AreaShape_Area',
 'AreaShape_Center_X': 'AreaShape_Center',
 'AreaShape_Center_Y': 'AreaShape_Center',
 'AreaShape_Center_Z': 'AreaShape_Center',
 'AreaShape_Compactness_x': 'AreaShape_Compactness',
 'AreaShape_Eccentricity_x': 'AreaShape_Eccentricity',
 'AreaShape_EulerNumber_x': 'AreaShape_EulerNumber',
 'AreaShape_Extent_x': 'AreaShape_Extent',
 'AreaShape_FormFactor_x': 'AreaShape_FormFactor',
 'AreaShape_MajorAxisLength_x': 'AreaShape_MajorAxisLength',
 'AreaShape_MaxFeretDiameter_x': 'AreaShape_MaxFeretDiameter',
 'AreaShape_MaximumRadius_x': 'AreaShape_MaximumRadius',
 'AreaShape_MeanRadius_x': 'AreaShape_MeanRadius',
 'AreaShape_MedianRadius_x': 'AreaShape_MedianRadius',
 'AreaShape_MinFeretDiameter_x': 'AreaShape_MinFeretDiameter',
 'AreaShape_MinorAxisLength_x': 'AreaShape_MinorAxisLength',
 'AreaShape_Orientation_x': 'AreaShape_Orientation',
 'AreaShape_Perimeter_x': 'AreaShape_Perimeter',
 'AreaShape_Solidity_x': 'AreaShape_Solidity'}

In [19]:
df_final = df_jnk.rename(columns = rename_dict)

In [21]:
list(df_final)

['ImageNumber',
 'ObjectNumber',
 'AreaShape_Area',
 'AreaShape_Center',
 'AreaShape_Center',
 'AreaShape_Center',
 'AreaShape_Compactness',
 'AreaShape_Eccentricity',
 'AreaShape_EulerNumber',
 'AreaShape_Extent',
 'AreaShape_FormFactor',
 'AreaShape_MajorAxisLength',
 'AreaShape_MaxFeretDiameter',
 'AreaShape_MaximumRadius',
 'AreaShape_MeanRadius',
 'AreaShape_MedianRadius',
 'AreaShape_MinFeretDiameter',
 'AreaShape_MinorAxisLength',
 'AreaShape_Orientation',
 'AreaShape_Perimeter',
 'AreaShape_Solidity',
 'Intensity_IntegratedIntensityEdge_dapi',
 'Intensity_IntegratedIntensity_dapi',
 'Intensity_LowerQuartileIntensity_dapi',
 'Intensity_MADIntensity_dapi',
 'Intensity_MassDisplacement_dapi',
 'Intensity_MaxIntensityEdge_dapi',
 'Intensity_MaxIntensity_dapi',
 'Intensity_MeanIntensityEdge_dapi',
 'Intensity_MeanIntensity_dapi',
 'Intensity_MedianIntensity_dapi',
 'Intensity_MinIntensityEdge_dapi',
 'Intensity_MinIntensity_dapi',
 'Intensity_StdIntensityEdge_dapi',
 'Intensity_StdI

In [25]:
len(df_jnk)

376

In [37]:
df_whole = df_whole[[ 'AreaShape_Area',
 'AreaShape_Center_X',
 'AreaShape_Center_Y',
 'AreaShape_Center_Z',
 'AreaShape_Compactness',
 'AreaShape_Eccentricity',
 'AreaShape_EulerNumber',
 'AreaShape_Extent',
 'AreaShape_FormFactor',
 'AreaShape_MajorAxisLength',
 'AreaShape_MaxFeretDiameter',
 'AreaShape_MaximumRadius',
 'AreaShape_MeanRadius',
 'AreaShape_MedianRadius',
 'AreaShape_MinFeretDiameter',
 'AreaShape_MinorAxisLength',
 'AreaShape_MinorAxisLength',
 'AreaShape_Perimeter',
 'AreaShape_Solidity',
 'Intensity_IntegratedIntensityEdge_dapi',
 'Intensity_IntegratedIntensity_dapi',
 'Intensity_LowerQuartileIntensity_dapi',
 'Intensity_MADIntensity_dapi',
 'Intensity_MassDisplacement_dapi',
 'Intensity_MaxIntensityEdge_dapi',
 'Intensity_MaxIntensity_dapi',
 'Intensity_MeanIntensityEdge_dapi',
 'Intensity_MeanIntensity_dapi',
 'Intensity_MedianIntensity_dapi',
 'Intensity_MinIntensityEdge_dapi',
 'Intensity_MinIntensity_dapi',
 'Intensity_StdIntensityEdge_dapi',
 'Intensity_StdIntensity_dapi',
 'Intensity_UpperQuartileIntensity_dapi',
 'Location_CenterMassIntensity_X_dapi',
 'Location_CenterMassIntensity_Y_dapi',
 'Location_CenterMassIntensity_Z_dapi',
 'Location_Center_X',
 'Location_Center_Y',
 'Location_Center_Z',
 'Location_MaxIntensity_X_dapi',
 'Location_MaxIntensity_Y_dapi',
 'Location_MaxIntensity_Z_dapi',
 'in_object']]

In [38]:
list(df_whole)

['AreaShape_Area',
 'AreaShape_Center_X',
 'AreaShape_Center_Y',
 'AreaShape_Center_Z',
 'AreaShape_Compactness',
 'AreaShape_Eccentricity',
 'AreaShape_EulerNumber',
 'AreaShape_Extent',
 'AreaShape_FormFactor',
 'AreaShape_MajorAxisLength',
 'AreaShape_MaxFeretDiameter',
 'AreaShape_MaximumRadius',
 'AreaShape_MeanRadius',
 'AreaShape_MedianRadius',
 'AreaShape_MinFeretDiameter',
 'AreaShape_MinorAxisLength',
 'AreaShape_MinorAxisLength',
 'AreaShape_MinorAxisLength',
 'AreaShape_MinorAxisLength',
 'AreaShape_Perimeter',
 'AreaShape_Solidity',
 'Intensity_IntegratedIntensityEdge_dapi',
 'Intensity_IntegratedIntensity_dapi',
 'Intensity_LowerQuartileIntensity_dapi',
 'Intensity_MADIntensity_dapi',
 'Intensity_MassDisplacement_dapi',
 'Intensity_MaxIntensityEdge_dapi',
 'Intensity_MaxIntensity_dapi',
 'Intensity_MeanIntensityEdge_dapi',
 'Intensity_MeanIntensity_dapi',
 'Intensity_MedianIntensity_dapi',
 'Intensity_MinIntensityEdge_dapi',
 'Intensity_MinIntensity_dapi',
 'Intensity_Std

In [25]:
os.chdir(r'C:\Users\timdaugird\Google Drive\legant_lab\_projects\200217_adipogen_zstacks\cell_profiler_output\dcts_output')

In [26]:
df_final.to_csv('data_for_nb_training.csv')

In [25]:
df_final.head()

,ImageNumber,ObjectNumber,AreaShape_Area,AreaShape_Center,AreaShape_Center,AreaShape_Center,AreaShape_Compactness,AreaShape_Eccentricity,AreaShape_EulerNumber,AreaShape_Extent,...,Location_CenterMassIntensity_X_dapi,Location_CenterMassIntensity_Y_dapi,Location_CenterMassIntensity_Z_dapi,Location_Center_X,Location_Center_Y,Location_Center_Z,Location_MaxIntensity_X_dapi,Location_MaxIntensity_Y_dapi,Location_MaxIntensity_Z_dapi,in_object
0,1,1,8331,282.0,0.0,1.0,2.352487,0.968781,1.0,0.658577,...,280.604103,18.974834,0.0,279.505582,20.227464,0,351.0,4.0,0.0,0.0
1,1,2,49529,2043.0,89.0,1.0,1.809739,0.919641,1.0,0.578779,...,1976.412703,155.525121,0.0,1972.332613,160.861556,0,2043.0,376.0,0.0,0.0
2,1,3,54217,1186.0,186.0,1.0,1.018289,0.560011,1.0,0.755912,...,1184.478240,188.679428,0.0,1185.255252,186.132394,0,1121.0,226.0,0.0,1.0
3,1,4,46013,455.0,474.0,1.0,1.013055,0.511901,1.0,0.776630,...,457.863619,470.246954,0.0,457.088214,468.432573,0,511.0,513.0,0.0,1.0
4,1,5,38938,1336.0,473.0,1.0,1.025519,0.598589,1.0,0.749168,...,1346.701876,470.424959,0.0,1344.709461,465.271765,0,1286.0,531.0,0.0,1.0


In [39]:
len(df_jnk)

376

### I would as well like to figure out if this classifier is doing better than just my original cutoffs:
area > 20000
eccentricity > .55

In [36]:
'''make a df according to original parameters'''
df_parsed = df_whole.loc[df_whole.AreaShape_Area > 20000]
df_parsed = df_parsed.loc[df_parsed.AreaShape_Eccentricity > .55]
'''make the opposite dataframe'''
df_unparsed = df_whole[~df_whole.isin(df_parsed)].dropna()

In [52]:
'''determing the false positive'''
fp = np.sum(np.subtract(np.ones(len(df_parsed)), df_parsed.in_object)) #false positive
fp

46.0

In [54]:
'''determine false negatives'''
fn = np.sum(df_unparsed.in_object)
fn

115.0

In [51]:
'''first thing will be to figure out accuracy according to tp + tn / tp + tn + fp + fn'''
one_array = np.ones(len(df_parsed))
tp = np.sum(df_parsed.in_object) # true positives
tn = np.sum(np.subtract(np.ones(len(df_unparsed)), df_unparsed.in_object)) #true negatives basically this is summation of all of the zeros
accur = (tp + tn) / (tp + tn + fp + fn)
accur

0.5718085106382979

In [50]:
tp / (tp + fp)

0.7051282051282052

In [43]:
tp #true positive

110.0

In [44]:
tn #true negative

105.0

46.0

115.0

In [ ]:
tp / (tp + )

In [27]:
df_whole.head()

,Unnamed: 0,AreaShape_Area_x,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_Center_Z,AreaShape_Compactness_x,AreaShape_Eccentricity_x,AreaShape_EulerNumber_x,AreaShape_Extent_x,AreaShape_FormFactor_x,...,Location_CenterMassIntensity_X_dapi,Location_CenterMassIntensity_Y_dapi,Location_CenterMassIntensity_Z_dapi,Location_Center_X,Location_Center_Y,Location_Center_Z,Location_MaxIntensity_X_dapi,Location_MaxIntensity_Y_dapi,Location_MaxIntensity_Z_dapi,in_object
0,2,54217,1186.0,186.0,1.0,1.018289,0.560011,1.0,0.755912,0.883736,...,1184.478240,188.679428,0.0,1185.255252,186.132394,0,1121.0,226.0,0.0,1.0
1,3,46013,455.0,474.0,1.0,1.013055,0.511901,1.0,0.776630,0.879143,...,457.863619,470.246954,0.0,457.088214,468.432573,0,511.0,513.0,0.0,1.0
2,4,38938,1336.0,473.0,1.0,1.025519,0.598589,1.0,0.749168,0.878855,...,1346.701876,470.424959,0.0,1344.709461,465.271765,0,1286.0,531.0,0.0,1.0
3,5,37143,1638.0,603.0,1.0,1.001414,0.195747,1.0,0.789051,0.893726,...,1638.702575,602.238092,0.0,1639.836362,603.160676,0,1589.0,640.0,0.0,1.0
4,6,41741,1129.0,702.0,1.0,1.015753,0.543809,1.0,0.774416,0.881953,...,1134.320520,703.623501,0.0,1134.146163,703.708320,0,1204.0,739.0,0.0,1.0


In [ ]:
df_parsed = df_whole.loc[df.AreaShape_Area_x]

### the rest is some bullshit that I had to trudge through in before figuring out just how easy this actually was. It was one of those moments where I got frustrated thinking that it shouldn't be so hard to do something. Turns out it was not anywhere near as hard as I was making it out to be

In [ ]:
train_cats = ['AreaShape_Area',
 'AreaShape_Compactness',
 'AreaShape_Eccentricity',
 'AreaShape_EulerNumber',
 'AreaShape_Extent',
 'AreaShape_FormFactor',
 'AreaShape_MajorAxisLength',
 'AreaShape_MaxFeretDiameter',
 'AreaShape_MaximumRadius',
 'AreaShape_MeanRadius',
 'AreaShape_MedianRadius',
 'AreaShape_MinFeretDiameter',
 'AreaShape_MinorAxisLength',
 'AreaShape_Orientation',
 'AreaShape_Perimeter',
 'AreaShape_Solidity']

In [67]:
from sklearn.datasets import load_iris

In [81]:
train_data = np.array([np.array(df[train_cats[0]]),
                       np.array(df[train_cats[1]]),
                       np.array(df[train_cats[2]]),
                       np.array(df[train_cats[3]]),
                       np.array(df[train_cats[4]]),
                       np.array(df[train_cats[5]]),
                       np.array(df[train_cats[6]]),
                       np.array(df[train_cats[7]]),
                       np.array(df[train_cats[8]]),
                       np.array(df[train_cats[9]]),
                       np.array(df[train_cats[10]]),
                       np.array(df[train_cats[11]]),
                       np.array(df[train_cats[12]]),
                       np.array(df[train_cats[13]]),
                       np.array(df[train_cats[14]]),
                       np.array(df[train_cats[15]])])


In [83]:
train_data.shape

(16, 17)

In [78]:
train_cats[15]

'AreaShape_Solidity'

In [ ]:
gnb = Gau

In [11]:
from sklearn import datasets

In [12]:
wine = datasets.load_wine()

In [33]:
gn

AttributeError: ndim

In [41]:
wine.data.shape

(178, 13)

In [71]:
wine.data.shape

(178, 13)

In [85]:
wine.target.shape

(178,)

In [86]:
y.shape

(17,)

In [158]:
os.chdir(r'C:\Users\timdaugird\Google Drive\legant_lab\_projects\200217_adipogen_zstacks\cell_profiler_output\dcts_output')

In [159]:
df_whole.to_csv('data_for_nb_training.csv')

In [160]:
model

GaussianNB(priors=None, var_smoothing=1e-09)

In [162]:
mod = model.fit(x_train, y_train)

In [171]:
(model.get_params())

{'priors': None, 'var_smoothing': 1e-09}

In [65]:
df_whole['AreaShape_MinorAxisLength.1'] == df_whole['AreaShape_MinorAxisLength.2']

0      False
1      False
2      False
3      False
4      False
       ...  
371    False
372    False
373    False
374    False
375    False
Length: 376, dtype: bool

In [66]:
df_whole['AreaShape_MinorAxisLength.1']

0     -58.060958
1      87.324161
2     -42.679831
3     -13.256940
4      22.393151
         ...    
371   -89.277824
372    18.844186
373   -27.338654
374   -37.278933
375     5.704488
Name: AreaShape_MinorAxisLength.1, Length: 376, dtype: float64

In [67]:
df_whole['AreaShape_MinorAxisLength.2']

0      238.697316
1      224.029428
2      198.948844
3      215.040300
4      210.783407
          ...    
371     27.894660
372    180.784048
373    192.339789
374    258.449122
375    162.795758
Name: AreaShape_MinorAxisLength.2, Length: 376, dtype: float64

### I am going to once again try and pull the appropriate data out of the original dataframe

In [7]:
'''pull the image number and object number from each'''

obj_numb = df_whole.ObjectNumber
img_numb = df_whole.ImageNumber

In [8]:
'''read in the original data set'''
os.chdir(r'C:\Users\timdaugird\Google Drive\legant_lab\_projects\200217_adipogen_zstacks\cell_profiler_output\dcts_output')
df_orig = pd.read_csv('MyExpt_nuclei.csv')

In [9]:
'''I think that the object number shoule be unique for each entry'''
len(df_orig)

5541

In [10]:
len(np.unique(df_orig.ObjectNumber))

47

In [23]:
df_whole

,Unnamed: 0,ImageNumber,ObjectNumber,AreaShape_Area,AreaShape_Center,AreaShape_Center.1,AreaShape_Center.2,AreaShape_Compactness,AreaShape_Eccentricity,AreaShape_EulerNumber,...,Location_CenterMassIntensity_X_dapi,Location_CenterMassIntensity_Y_dapi,Location_CenterMassIntensity_Z_dapi,Location_Center_X,Location_Center_Y,Location_Center_Z,Location_MaxIntensity_X_dapi,Location_MaxIntensity_Y_dapi,Location_MaxIntensity_Z_dapi,in_object
0,0,1,1,8331,282.0,0.0,1.0,2.352487,0.968781,1.0,...,280.604103,18.974834,0.0,279.505582,20.227464,0,351.0,4.0,0.0,0.0
1,1,1,2,49529,2043.0,89.0,1.0,1.809739,0.919641,1.0,...,1976.412703,155.525121,0.0,1972.332613,160.861556,0,2043.0,376.0,0.0,0.0
2,2,1,3,54217,1186.0,186.0,1.0,1.018289,0.560011,1.0,...,1184.478240,188.679428,0.0,1185.255252,186.132394,0,1121.0,226.0,0.0,1.0
3,3,1,4,46013,455.0,474.0,1.0,1.013055,0.511901,1.0,...,457.863619,470.246954,0.0,457.088214,468.432573,0,511.0,513.0,0.0,1.0
4,4,1,5,38938,1336.0,473.0,1.0,1.025519,0.598589,1.0,...,1346.701876,470.424959,0.0,1344.709461,465.271765,0,1286.0,531.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,371,119,5,43660,684.0,777.0,1.0,1.020885,0.562426,1.0,...,682.957034,787.376642,0.0,682.833646,787.113468,0,736.0,834.0,0.0,1.0
372,372,119,6,73024,1242.0,1104.0,1.0,1.094695,0.713009,1.0,...,1262.052340,1084.101227,0.0,1274.157702,1073.073277,0,1183.0,1142.0,0.0,0.0
373,373,119,7,24606,123.0,1485.0,1.0,1.003381,0.302988,1.0,...,123.863960,1484.220579,0.0,123.851865,1483.026213,0,182.0,1499.0,0.0,1.0
374,374,119,8,42792,1826.0,1695.0,1.0,1.348277,0.882464,1.0,...,1760.025239,1685.401802,0.0,1757.533394,1686.956230,0,1751.0,1674.0,0.0,0.0


In [30]:
df_whole_.sort_values(['ImageNumber', 'ObjectNumber'], inplace = True)

In [31]:
df_whole.AreaShape_Center == 

0       282.0
1      2043.0
2      1186.0
3       455.0
4      1336.0
        ...  
371     684.0
372    1242.0
373     123.0
374    1826.0
375    1165.0
Name: AreaShape_Center, Length: 376, dtype: float64